In [1]:
# mount google drive
from google.colab import drive
import os
drive.mount('/content/drive')


notebook_dir = '/content/drive/My Drive/Lighthouse Labs/LLM_Project_Sentiment_Analysis'
os.chdir(notebook_dir)

Mounted at /content/drive


In [2]:
!pip install transformers datasets torch accelerate evaluate wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
# import required module
from datasets import load_dataset

# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
from nltk.stem import WordNetLemmatizer
from transformers import pipelines
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch
import evaluate
from datasets import load_dataset


nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
# funcitons

# function to clean text - removes punctuation, converts to lowercase, removes stop words
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
    text = text.replace("\n", " ")  # Remove newlines

    # Remove stop words and lemmatize
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

# Define text preprocessing function for batches
def preprocess_text_pretrained(batch):
    cleaned_texts = []
    for text in batch["text"]:  # Loop through each text in the batch
        text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
        text = text.lower()  # Convert to lowercase
        text = re.sub(r"\d+", "", text)  # Remove numbers
        text = text.replace("\n", " ")  # Remove newlines
        cleaned_texts.append(text)
    return {"text": cleaned_texts}  # Return a dictionary

# Tokenization function
def tokenize_function(examples):
    examples["text"] = preprocess_text(examples["text"])  # Clean text
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Function to preprocess & tokenize each dataset
def preprocess_and_tokenize(example):
    cleaned_text = preprocess_text(example["text"])  # Preprocess text
    tokenized_output = tokenizer(cleaned_text, padding="max_length", truncation=True)  # Tokenize

    return {
        "input_ids": tokenized_output["input_ids"],
        "attention_mask": tokenized_output["attention_mask"],
        "label": example["label"]
    }

# Function to compute accuracy
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)  # Convert logits to class labels
    return accuracy.compute(predictions=predictions, references=labels)

In [5]:
# load dataset for sentiment analysis movie rating imdb
ds = load_dataset('imdb')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
# available splits for imdb dataset
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [7]:
# example of the data
ds['train'][0]

#returns label only
#ds['train'][0]['label']

#returns text only
#ds['train'][0]['text']

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [8]:
# data features
ds['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [9]:
# dataset to dataframe
df_train = pd.DataFrame(ds['train'])
df_test = pd.DataFrame(ds['test'])

#retrieving the first row to see the data - has <br>
#first_row = df_train['text'].iloc[0]
#print(first_row)

In [10]:
from hashlib import new
# convert dataframe to dataset again to be used to work with hugging api
from datasets import Dataset, DatasetDict
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

# reconstruct both datasets into a Dataset Dict obj
new_ds = DatasetDict()
new_ds['train'] = ds_train
new_ds['test'] = ds_test

new_ds


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [11]:
#---- Text Preprocessing ----#


# convert datasets to dataframe
df_train = new_ds['train'].to_pandas()
df_test = new_ds['test'].to_pandas()

stop_words = set(stopwords.words('english'))

# Apply preprocessing to both train and test
new_ds['train'] = new_ds['train'].map(lambda x: {'text': preprocess_text(x['text']), 'label': x['label']})
new_ds['test'] = new_ds['test'].map(lambda x: {'text': preprocess_text(x['text']), 'label': x['label']})

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [12]:
# initialize and fit TF_IDF vectorize on training data
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(new_ds['train']['text'])
X_test_tfidf = tfidf_vectorizer.transform(new_ds['test']['text'])

Question
How many documents are in your dataset? How many total words make up the vocabulary in your dataset?

In [13]:
# number of documents
num_documents_train = len(new_ds['train'])
print(f"Number of documents in train dataset: {num_documents_train}") #25000
num_documents_test = len(new_ds['test'])
print(f"Number of documents in test dataset: {num_documents_test}")

# vocabulary size
vocab_size = len(tfidf_vectorizer.get_feature_names_out())
print(f"Total number of unique words in vocabulary: {vocab_size}") #120611

Number of documents in train dataset: 25000
Number of documents in test dataset: 25000
Total number of unique words in vocabulary: 109030


In [14]:
# prepare target variables
y_train = new_ds['train']['label']
y_test = new_ds['test']['label']

In [15]:
# train Logistic Regression model

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_tfidf, y_train)

# predict on test set
y_pred = classifier.predict(X_test_tfidf)

# evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88     12500
           1       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



Reflection Questions: How well does this model perform? What are some of the limitations you can see from this model you have created?

In [16]:
# Class 0: Represents negative sentiment reviews.
# Class 1: Represents positive sentiment reviews.
# Support: The number of actual occurrences of each class (12,500 for both positive and negative reviews).
# Accuracy: 88% of all predictions were correct.

Applying Pre-trained Model

In [17]:
# Specify the model name from Hugging Face
model_name = "distilbert-base-uncased-finetuned-sst-2-english"  # Pretrained for sentiment analysis

# Download and load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [18]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


Device set to use cuda:0


In [19]:
# test the pre-trained model
text = "boring"
result = classifier(text)

print(result)

[{'label': 'NEGATIVE', 'score': 0.9997890591621399}]


In [20]:
# save model locally
model.save_pretrained("./huggingface_model")
tokenizer.save_pretrained("./huggingface_model")

('./huggingface_model/tokenizer_config.json',
 './huggingface_model/special_tokens_map.json',
 './huggingface_model/vocab.txt',
 './huggingface_model/added_tokens.json',
 './huggingface_model/tokenizer.json')

In [21]:
# load the model locally
local_path = "./huggingface_model"

model = AutoModelForSequenceClassification.from_pretrained(local_path)
tokenizer = AutoTokenizer.from_pretrained(local_path)

In [22]:
ds_train_2 = Dataset.from_pandas(df_train)
ds_test_2= Dataset.from_pandas(df_test)

# reconstruct both datasets into a Dataset Dict obj
new_ds_pretrained = DatasetDict()
new_ds_pretrained['train'] = ds_train_2
new_ds_pretrained['test'] = ds_test_2

type(new_ds_pretrained['train'])

datasets.arrow_dataset.Dataset

In [23]:
# Apply processing and tokenization to train and test datasets
new_ds_pretrained["train"] = new_ds_pretrained["train"].map(preprocess_and_tokenize, remove_columns=["text"])
new_ds_pretrained["test"] = new_ds_pretrained["test"].map(preprocess_and_tokenize, remove_columns=["text"])


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
# print a sample from the train set
print(new_ds_pretrained["train"][0])

{'label': 0, 'input_ids': [101, 12524, 8025, 6672, 7174, 2860, 2678, 3573, 6704, 5129, 2034, 2207, 2036, 2657, 2034, 8243, 1057, 7661, 2412, 2699, 4607, 2406, 3568, 5470, 2143, 2641, 6801, 2428, 2156, 2870, 19892, 7987, 5436, 8857, 2105, 2402, 4467, 3689, 3076, 2315, 14229, 2215, 4553, 2673, 2166, 3327, 2215, 3579, 3086, 2437, 4066, 4516, 2779, 25430, 14728, 2245, 3056, 2576, 3277, 5148, 2162, 2679, 3277, 2142, 2110, 4851, 3761, 6623, 7939, 4697, 2078, 8947, 5448, 4331, 3348, 3689, 3836, 23175, 2496, 2273, 19892, 7987, 3102, 8025, 6672, 7174, 2860, 2095, 3283, 2641, 26932, 2428, 3348, 16371, 25469, 3496, 2521, 2130, 2915, 2066, 10036, 2135, 2081, 22555, 2080, 2406, 2386, 2568, 2424, 16880, 4507, 3348, 16371, 25469, 2350, 18785, 4467, 5988, 2130, 13749, 7849, 24544, 15835, 3437, 2204, 2214, 2879, 2198, 4811, 3348, 3496, 3152, 19892, 7987, 4012, 3549, 2094, 12127, 2755, 3348, 3491, 2143, 3491, 6018, 3800, 2738, 5213, 2111, 2191, 2769, 3491, 26932, 4258, 2637, 8025, 6672, 7174, 2860, 2204

In [ ]:
# fine tune

accuracy = evaluate.load("accuracy")

# setup training arguments
training_args = TrainingArguments(
    output_dir="./imdb_finetuned_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none"
)

#fine tune using trainer
os.environ["WANDB_DISABLED"] = "true"

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=new_ds_pretrained["train"],
    eval_dataset=new_ds_pretrained["test"],
    compute_metrics=compute_metrics
)

# Start training
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.259800,0.245967,0.906760
2,0.180100,0.265461,0.911400
3,0.116100,0.357930,0.911040


TrainOutput(global_step=4689, training_loss=0.19481058582412483, metrics={'train_runtime': 4957.0505, 'train_samples_per_second': 15.13, 'train_steps_per_second': 0.946, 'total_flos': 9935054899200000.0, 'train_loss': 0.19481058582412483, 'epoch': 3.0})

In [ ]:
# save fine tune model

# Save the model locally
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")

from google.colab import files
import shutil

shutil.make_archive("fine_tuned_distilbert", 'zip', "./fine_tuned_distilbert")
files.download("fine_tuned_distilbert.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# running prediction for an exmaple on the model

# Load fine-tuned model for predictions
sentiment_pipe = pipeline("sentiment-analysis", model="./fine_tuned_distilbert", tokenizer="./fine_tuned_distilbert")

# Example movie reviews
reviews = [
    "This movie was absolutely amazing! I loved every moment.",
    "I was really disappointed. The plot made no sense and the acting was bad.",
    "The cinematography was beautiful, but the story was predictable.",
    "I regret watching this film. Such a waste of time."
]

# Predict sentiment
results = sentiment_pipe(reviews)

# Print predictions
for review, result in zip(reviews, results):
    print(f"Review: {review}\nPredicted Sentiment: {result}\n")

Device set to use cuda:0


Review: This movie was absolutely amazing! I loved every moment.
Predicted Sentiment: {'label': 'POSITIVE', 'score': 0.9996134638786316}

Review: I was really disappointed. The plot made no sense and the acting was bad.
Predicted Sentiment: {'label': 'NEGATIVE', 'score': 0.9998395442962646}

Review: The cinematography was beautiful, but the story was predictable.
Predicted Sentiment: {'label': 'NEGATIVE', 'score': 0.9981245398521423}

Review: I regret watching this film. Such a waste of time.
Predicted Sentiment: {'label': 'NEGATIVE', 'score': 0.9998260140419006}



In [ ]:
# evaluating the model on the test data

metrics = trainer.evaluate()
print(metrics)  # Print accuracy and loss on the test set

{'eval_loss': 0.3579297661781311, 'eval_accuracy': 0.91104, 'eval_runtime': 413.2016, 'eval_samples_per_second': 60.503, 'eval_steps_per_second': 3.783, 'epoch': 3.0}


evaluation metrics:

{'eval_loss': 0.3579297661781311, 'eval_accuracy': 0.91104, 'eval_runtime': 413.2016, 'eval_samples_per_second': 60.503, 'eval_steps_per_second': 3.783, 'epoch': 3.0}

The model has 0.3579 evaluaiton loss and 91% accuracy

The model is performing well

Evaluating and Optimizing

In [24]:
# connecting to Huggingface
from huggingface_hub import notebook_login
notebook_login()


In [25]:
# install git large file storage
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [26]:
# locally loading the fine tuned model
model_finetuned= AutoModelForSequenceClassification.from_pretrained("./fine_tuned_distilbert")
tokenizer_finetuned = AutoTokenizer.from_pretrained("./fine_tuned_distilbert")

In [27]:
# modify and optimize hyperparameters for retraining
from transformers import TrainingArguments

repo_name = "imdb-finetuned-distilbert"  # Hugging Face model repo name

accuracy = evaluate.load("accuracy")

training_args_optimized = TrainingArguments(
    output_dir=repo_name,
    evaluation_strategy="epoch",
    learning_rate=3e-5,  # Increased LR to speed up learning
    per_device_train_batch_size=16,  # Larger batch if GPU allows
    per_device_eval_batch_size=16,
    num_train_epochs=5,  # More training passes
    weight_decay=0.02,  # Better regularization
    warmup_steps=500,  # Stabilizes learning in the beginning
    push_to_hub=True,  # Upload fine-tuned model to Hugging Face
    save_strategy="epoch",
    logging_strategy="epoch",
    report_to="none"  # Disable W&B logging
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
trainer = Trainer(
    model=model_finetuned,
    args=training_args_optimized,
    train_dataset=new_ds_pretrained["train"],
    eval_dataset=new_ds_pretrained["test"],
    tokenizer=tokenizer_finetuned,  # Add tokenizer
    compute_metrics=compute_metrics)

trainer.train()

<ipython-input-28-fb5dff065746>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.105000,0.486047,0.892680
2,0.084600,0.565210,0.894240
3,0.061400,0.489563,0.904320
4,0.034100,0.677959,0.910120
5,0.010900,0.741574,0.910200


No files have been modified since last commit. Skipping to prevent empty commit.


TrainOutput(global_step=7815, training_loss=0.05920260667953443, metrics={'train_runtime': 7578.1024, 'train_samples_per_second': 16.495, 'train_steps_per_second': 1.031, 'total_flos': 1.6558424832e+16, 'train_loss': 0.05920260667953443, 'epoch': 5.0})

In [29]:
model.push_to_hub("imdb-optimized-finetuned-distilbert")
tokenizer.push_to_hub("imdb-optimized-finetuned-distilbert")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dibajafarnejad/imdb-optimized-finetuned-distilbert/commit/c3c3739784b1d1aed9560c757ab8746850491e3b', commit_message='Upload tokenizer', commit_description='', oid='c3c3739784b1d1aed9560c757ab8746850491e3b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dibajafarnejad/imdb-optimized-finetuned-distilbert', endpoint='https://huggingface.co', repo_type='model', repo_id='dibajafarnejad/imdb-optimized-finetuned-distilbert'), pr_revision=None, pr_num=None)